In [153]:
import numpy as np
import pandas as pd


In [154]:
#importing excel files to pandas
ordersdf=pd.read_excel("/content/CompanyXOrderReport.xlsx")
pincodedf=pd.read_excel("/content/CompanyXPincodeZones.xlsx")
SKUmasterdf=pd.read_excel("/content/CompanyXSKUMaster.xlsx")
CourierCompinvoicedf=pd.read_excel("/content/CourierCompanyInvoice.xlsx")
couriercompratedf=pd.read_excel("/content/CourierCompanyRates.xlsx")


In [155]:
ordersdf.rename(columns={'ExternOrderNo':'Order ID'},inplace=True) #renaming columns for same column name
dummydf=pd.merge(ordersdf,SKUmasterdf,on='SKU') #merging ordersdf and SKUmasterdf

In [156]:
# calculating Totel Weight As Company X
dummydf['TotelWeightAsX']=dummydf['Order Qty']*dummydf['Weight (g)']

In [157]:
#getting essential columns
dummydf=dummydf[['Order ID','TotelWeightAsX']]
#group TotelWeightAsX by order id by sum(TotelWeightAsX)
dummydf = dummydf.groupby('Order ID').agg(TotelWeightAsX=('TotelWeightAsX', 'sum')).reset_index()


In [158]:
#Weight in kg
dummydf['TotelWeightAsX']=dummydf['TotelWeightAsX']/1000
dummydf['TotelWeightAsX'] = dummydf['TotelWeightAsX'].round(2)
len(dummydf)

124

In [159]:
#drop duplicate rows in pincodedf
pincodedf=pincodedf.drop_duplicates()


In [160]:
#merge with CourierCompinvoicedf by order id to get customer pincode
dummydf=dummydf.merge(CourierCompinvoicedf,on='Order ID')
dummydf=dummydf[['Order ID','TotelWeightAsX','Customer Pincode']]
dummydf

,Order ID,TotelWeightAsX,Customer Pincode
0,2001806210,0.22,140604
1,2001806226,0.48,723146
2,2001806229,0.50,421204
3,2001806232,1.30,507101
4,2001806233,0.24,263139
...,...,...,...
119,2001821995,0.48,342008
120,2001822466,1.38,342301
121,2001823564,0.67,492001
122,2001825261,1.56,517128


In [161]:
#merge with pincodedf to get zone
dummydf=dummydf.merge(pincodedf,on='Customer Pincode',how='left')

In [162]:
#getting essential columns
dummydf=dummydf[['Order ID','TotelWeightAsX','Customer Pincode','Zone']]


In [163]:
# convert Zone to lower case using apply method
couriercompratedf['Zone']=couriercompratedf['Zone'].apply(lambda x:x.lower())

In [164]:
CourierCompinvoicedf.head()

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.)
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4


In [165]:
#merge with CourierCompinvoicedf to get type of shipment
dummydf=dummydf.merge(CourierCompinvoicedf[['Order ID','Type of Shipment']],on='Order ID',how='left')

In [166]:
dummydf


,Order ID,TotelWeightAsX,Customer Pincode,Zone,Type of Shipment
0,2001806210,0.22,140604,b,Forward charges
1,2001806226,0.48,723146,d,Forward charges
2,2001806229,0.50,421204,d,Forward charges
3,2001806232,1.30,507101,d,Forward charges
4,2001806233,0.24,263139,b,Forward charges
...,...,...,...,...,...
119,2001821995,0.48,342008,b,Forward charges
120,2001822466,1.38,342301,b,Forward charges
121,2001823564,0.67,492001,d,Forward and RTO charges
122,2001825261,1.56,517128,d,Forward and RTO charges


In [167]:
coucomrate=couriercompratedf.to_dict()
coucomrate

{'Zone': {0: 'a', 1: 'b', 2: 'c', 3: 'd', 4: 'e'},
 'Weight Slabs': {0: 0.5, 1: 1.0, 2: 1.25, 3: 1.5, 4: 2.0},
 'Forward Fixed Charge': {0: 29.5, 1: 33.0, 2: 40.1, 3: 45.4, 4: 56.6},
 'Forward Additional Weight Slab Charge': {0: 23.6,
  1: 28.3,
  2: 38.9,
  3: 44.8,
  4: 55.5},
 'RTO Fixed Charge': {0: 13.6, 1: 20.5, 2: 31.9, 3: 41.3, 4: 50.7},
 'RTO Additional Weight Slab Charge': {0: 23.6,
  1: 28.3,
  2: 38.9,
  3: 44.8,
  4: 55.5}}

In [168]:
#function to calculate charge as per company X
def CalculateCharge(Zone,TotelWeightAsX,Type_of_Shipment):
  index=0
  for k,v in coucomrate['Zone'].items():
    #check for zone
    if v==Zone:
      index=k

  fwdfixcharge=coucomrate['Forward Fixed Charge'][index]
  rtofixcharge=coucomrate['RTO Fixed Charge'][index]
  if TotelWeightAsX < coucomrate['Weight Slabs'][index]:
    if Type_of_Shipment=="Forward charges":
      return fwdfixcharge
    elif Type_of_Shipment=='Forward and RTO charges':
      return fwdfixcharge+rtofixcharge
  else:
    x=2
    while x*TotelWeightAsX < coucomrate['Weight Slabs'][index]:
        x+=1
    if Type_of_Shipment=="Forward charges":
      return fwdfixcharge+(x-1)*coucomrate['Forward Additional Weight Slab Charge'][index]
    elif Type_of_Shipment=="Forward and RTO charges":
      return fwdfixcharge+(x-1)*coucomrate['Forward Additional Weight Slab Charge'][index]+rtofixcharge+(x-1)*coucomrate['RTO Additional Weight Slab Charge'][index]





In [169]:
#by using apply method and lambda function calling CalculateChargeFwd method
dummydf['ExpectedChargeAsX']=dummydf[['Zone','TotelWeightAsX','Type of Shipment']].apply(lambda x:CalculateCharge(*x),axis=1)

In [170]:
dummydf.rename(columns={'Zone':'ZoneAsX'},inplace=True)

In [171]:
dummydf=dummydf.merge(CourierCompinvoicedf,on='Order ID')

In [172]:
dummydf.head(2)

,Order ID,TotelWeightAsX,Customer Pincode_x,ZoneAsX,Type of Shipment_x,ExpectedChargeAsX,AWB Code,Charged Weight,Warehouse Pincode,Customer Pincode_y,Zone,Type of Shipment_y,Billing Amount (Rs.)
0,2001806210,0.22,140604,b,Forward charges,33.0,1091117221940,2.92,121003,140604,b,Forward charges,174.5
1,2001806226,0.48,723146,d,Forward charges,45.4,1091117222065,0.68,121003,723146,d,Forward charges,90.2


In [173]:
#using to_list method for getting dummydf columns list
collst=dummydf.columns.to_list()
collst

['Order ID',
 'TotelWeightAsX',
 'Customer Pincode_x',
 'ZoneAsX',
 'Type of Shipment_x',
 'ExpectedChargeAsX',
 'AWB Code',
 'Charged Weight',
 'Warehouse Pincode',
 'Customer Pincode_y',
 'Zone',
 'Type of Shipment_y',
 'Billing Amount (Rs.)']

In [174]:
dummydf=dummydf[['Order ID','AWB Code','TotelWeightAsX','ZoneAsX','ExpectedChargeAsX','Charged Weight','Zone','Billing Amount (Rs.)']]
dummydf.rename(columns={'Billing Amount (Rs.)':'Charges Billed by Courier Company (Rs.)'},inplace=True)

In [175]:
dummydf.head()

,Order ID,AWB Code,TotelWeightAsX,ZoneAsX,ExpectedChargeAsX,Charged Weight,Zone,Charges Billed by Courier Company (Rs.)
0,2001806210,1091117221940,0.22,b,33.0,2.92,b,174.5
1,2001806226,1091117222065,0.48,d,45.4,0.68,d,90.2
2,2001806229,1091117222080,0.50,d,45.4,0.71,d,90.2
3,2001806232,1091117222124,1.30,d,45.4,1.30,d,135.0
4,2001806233,1091117222135,0.24,b,33.0,0.78,b,61.3


In [176]:
#calculate diff between ExpectedChargeAsX and Charges Billed by Courier Company (Rs.)
dummydf['DiffExpectedandBilled']=dummydf['ExpectedChargeAsX']-dummydf['Charges Billed by Courier Company (Rs.)']

In [177]:
dummydf.head()

,Order ID,AWB Code,TotelWeightAsX,ZoneAsX,ExpectedChargeAsX,Charged Weight,Zone,Charges Billed by Courier Company (Rs.),DiffExpectedandBilled
0,2001806210,1091117221940,0.22,b,33.0,2.92,b,174.5,-141.5
1,2001806226,1091117222065,0.48,d,45.4,0.68,d,90.2,-44.8
2,2001806229,1091117222080,0.50,d,45.4,0.71,d,90.2,-44.8
3,2001806232,1091117222124,1.30,d,45.4,1.30,d,135.0,-89.6
4,2001806233,1091117222135,0.24,b,33.0,0.78,b,61.3,-28.3


In [178]:
#gettting a deep copy of dataframe
OutputData1=dummydf.copy()

In [179]:
# counts() function count over rate under rate bills and sum also
def countandsum(DiffExpectedandBilled):
  countzero=0
  sumamtzero=0
  countpos=0
  sumamtpos=0
  countneg=0
  sumamtneg=0
  for i in DiffExpectedandBilled:
    if i == 0:
      countzero+=1
      sumamtzero+=i
    elif i < 0:
      countneg+=1
      sumamtneg+=i
    else:
      countpos+=1
      sumamtpos+=i
  return {'countzero':countzero,'countpos':countpos,'countneg':countneg,'sumamtzero':sumamtzero,'sumamtpos':sumamtpos,'sumamtneg':sumamtneg}


In [180]:
DiffExpectedandBilled=dummydf['DiffExpectedandBilled'].to_list()
resultset=countandsum(DiffExpectedandBilled)


In [181]:
resultset

{'countzero': 7,
 'countpos': 3,
 'countneg': 114,
 'sumamtzero': 0.0,
 'sumamtpos': 47.20000000000002,
 'sumamtneg': -7797.099999999994}

In [182]:
#total billing amount
Totalinvoicecharge=sum(dummydf['Charges Billed by Courier Company (Rs.)'])

In [183]:
#summary dict
summary={'':['Total orders where X has been correctly charged','Total Orders where X has been overcharged',
             'Total Orders where X has been undercharged'],'Count':[resultset['countzero'],resultset['countneg'],resultset['countpos']],
         'Amount (Rs.)':[Totalinvoicecharge,-resultset['sumamtneg'],resultset['sumamtpos']]
         }

In [184]:
OutputData2=pd.DataFrame(summary)

In [185]:
OutputData2

,,Count,Amount (Rs.)
0,Total orders where X has been correctly charged,7,13648.2
1,Total Orders where X has been overcharged,114,7797.1
2,Total Orders where X has been undercharged,3,47.2


In [186]:
OutputData1

,Order ID,AWB Code,TotelWeightAsX,ZoneAsX,ExpectedChargeAsX,Charged Weight,Zone,Charges Billed by Courier Company (Rs.),DiffExpectedandBilled
0,2001806210,1091117221940,0.22,b,33.0,2.92,b,174.5,-141.5
1,2001806226,1091117222065,0.48,d,45.4,0.68,d,90.2,-44.8
2,2001806229,1091117222080,0.50,d,45.4,0.71,d,90.2,-44.8
3,2001806232,1091117222124,1.30,d,45.4,1.30,d,135.0,-89.6
4,2001806233,1091117222135,0.24,b,33.0,0.78,b,61.3,-28.3
...,...,...,...,...,...,...,...,...,...
119,2001821995,1091121183730,0.48,b,33.0,0.50,d,45.4,-12.4
120,2001822466,1091121305541,1.38,b,61.3,1.10,d,135.0,-73.7
121,2001823564,1091121666133,0.67,d,86.7,0.70,d,172.8,-86.1
122,2001825261,1091121981575,1.56,d,176.3,1.60,d,345.0,-168.7


In [187]:
#data frame to excel file
OutputData1.to_excel('OutputData1.xlsx', index=False)

In [188]:

OutputData2.to_excel('OutputData2.xlsx', index=False)